In [3]:
%pip install pandas
%pip install holidays
%pip install numpy
%pip install --no-cache-dir --force-reinstall xgboost
%pip install -U scikit-learn

You should consider upgrading via the '/Users/junkyunglee/Documents/Projects/cxc-2025/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/junkyunglee/Documents/Projects/cxc-2025/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/junkyunglee/Documents/Projects/cxc-2025/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 1.9 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 14.5 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstal

In [4]:
import pandas as pd
import numpy as np
import pickle
import requests
from datetime import timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Data Preparation

In [5]:
df_bills = pd.read_csv('data/bills.csv')
df_venues = pd.read_csv('data/venues.csv')

In [6]:
df_venues.columns

Index(['venue_xref_id', 'concept', 'city', 'country', 'start_of_day_offset'], dtype='object')

In [7]:
df_venues = df_venues.drop(columns='start_of_day_offset')
df_venues

,venue_xref_id,concept,city,country
0,ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9...,FAST_FOOD,Albuquerque,US
1,e7cec41c46a9706ba1a702b368be5431913b6dc9c9da49...,FAMILY_DINING,Brooklyn,US
2,e63e79791883b0c4ac71d41eaee727932c13d6a4ec8f78...,FAST_CASUAL,Windsor,CA
3,c100388c9328b30b6bdfbea43f113d367ae9d32b03e84b...,BAR,Burnaby,CA
4,a819ee7f83f27dd3d7f44515e22aca3c26e6704ddd2a9b...,FAMILY_DINING,Tisdale,CA
...,...,...,...,...
596,cfd1fa6e8f122e93b1cb02e7b7541749335e7088095f8e...,NaN,Toronto,CA
597,75e7251298f2d62406dbfe1a11011cffdad90b27704687...,NaN,Richardson,US
598,40c38939612f59ad418569e94a075935576bff0449c7f8...,FAST_CASUAL,Guelph,CA
599,2ed5bf26505a6064d09b40bacdb6fccc9665c3de029494...,NaN,Mississauga,CA


## Data Cleaning

In [8]:
missing_venues = df_venues[~df_venues['venue_xref_id'].isin(df_bills['venue_xref_id'])]
missing_venues

# All 'venue_xref_id' in df_venues exist in 'venue_xref_id' column in df_bills

,venue_xref_id,concept,city,country


In [9]:
print(df_venues['concept'].unique())

# We will omit fine_dining restaurants since they don't do promotions
# We will omit pop_up restaurants since they don't have long-term data
concept_counts = df_venues['concept'].value_counts(dropna=False)
filtered_counts = concept_counts.loc[['FINE_DINING', 'POP_UP']]
nan_count = df_venues['concept'].isna().sum()
filtered_counts.loc['NaN'] = nan_count
print(filtered_counts)

['FAST_FOOD' 'FAMILY_DINING' 'FAST_CASUAL' 'BAR' 'CAFE' 'SPORTS_CLUB' nan
 'FINE_DINING' 'BREWERY' 'POP_UP' 'BAKERY' 'BUFFET' 'HOTEL'
 'ENTERTAINMENT_COMPLEX' 'FOOD_TRUCK']
concept
FINE_DINING     19
POP_UP           1
NaN            136
Name: count, dtype: int64


In [10]:
print(df_venues['city'].isna().sum())
# No NaN in city column

0


In [11]:
# Get venue_xref_id for rows where concept is 'FINE_DINING', 'POP_UP', or NaN
venues_to_remove = df_venues[df_venues['concept'].isin(['FINE_DINING', 'POP_UP']) | df_venues['concept'].isna()]['venue_xref_id'].unique()
# Keep only rows where venue_xref_id is NOT in venues_to_remove
df_venues = df_venues[~df_venues['venue_xref_id'].isin(venues_to_remove)]
# Remove rows from df_bills that have the filtered venue_xref_id
df_bills = df_bills[~df_bills['venue_xref_id'].isin(venues_to_remove)]
print(f"Filtered df_venues: {df_venues.shape}")
print(f"Filtered df_bills: {df_bills.shape}")


Filtered df_venues: (445, 4)
Filtered df_bills: (7121732, 20)


In [12]:
df_bills.columns

Index(['bill_paid_at_local', 'bill_total_billed',
       'bill_total_discount_item_level', 'bill_total_gratuity',
       'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'bill_uuid',
       'business_date', 'order_duration_seconds', 'order_seated_at_local',
       'order_closed_at_local', 'order_take_out_type_label', 'order_uuid',
       'payment_amount', 'payment_count', 'payment_total_tip',
       'sales_revenue_with_tax', 'venue_xref_id', 'waiter_uuid'],
      dtype='object')

In [13]:
df_merged = df_bills.merge(df_venues, on="venue_xref_id", how="left")
df_merged


,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,...,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid,concept,city,country
0,2024-07-02 20:00:53,102.53,0.00,0.0,95.00,7.53,0.0,240702200053~8792ADCC-545E-4AF3-9836-9C428ED8285C,2024-07-02,5071,...,240702183622~4B0A3D27-501D-4C94-BEBC-7B7F3C91A0F4,124.53,1,22.0,102.53,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,200421150750~37D0C51E-EC4F-4EA1-B549-D223DA183ABD,BAR,Grand Forks,US
1,2024-07-03 21:43:39,5.38,5.00,0.0,5.00,0.38,0.0,240703214339~B06B2A2F-B1A0-44C0-920A-242F41B58BD2,2024-07-03,6968,...,240703194731~40651D6D-4A16-4889-8C12-0F9A00D8BCF7,5.38,1,0.0,5.38,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
2,2024-07-03 18:23:56,141.90,0.00,0.0,132.00,9.90,0.0,240703182356~D082FE98-841C-4EAB-B15E-BEA41FA1CDF0,2024-07-03,2011,...,240703175025~0AEB61B0-220A-45A3-807E-78A1F432BB4C,171.90,1,30.0,141.90,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
3,2024-07-03 19:33:21,63.77,0.00,0.0,59.00,4.77,0.0,240703193321~273CBB07-FAB0-49B7-A445-8474F9A4570F,2024-07-03,5480,...,240703180201~4B276496-BE92-4DE6-BD75-565929827014,73.77,1,10.0,63.77,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
4,2024-07-05 21:01:14,210.65,0.00,0.0,195.00,15.65,0.0,240705210114~D423B906-1AEE-4DB8-84BA-234267A14E05,2024-07-05,4773,...,240705194141~F90FEB4F-D238-449E-B476-D16966DF4EEC,252.65,1,42.0,210.65,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7121727,2024-12-29 20:32:54,82.42,0.00,0.0,82.42,0.00,0.0,241229203254~464AA06D-F986-4C51-8A53-A7B8362072D6,2024-12-29,4382,...,241229191952~96F91DE2-5ECC-4AC9-9228-B0741F3494B1,82.42,1,0.0,82.42,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,220831171122~1455DBA8-211C-4281-BF27-8698365CA3EB,FAMILY_DINING,Edmonton,CA
7121728,2024-12-30 20:36:19,87.92,0.00,0.0,87.92,0.00,0.0,241230203619~37348A86-98D1-4113-8EDD-06A3CF61A85B,2024-12-30,5244,...,241230190855~BB4604F2-B7BA-487A-99AF-7AEB77CA3EEC,87.92,1,0.0,87.92,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA
7121729,2024-12-31 14:24:23,51.96,0.00,0.0,51.96,0.00,0.0,241231142423~9D2E9295-9E59-4E76-8242-D230C9FD2494,2024-12-31,963,...,241231140820~4D531668-3D68-4789-A1DB-BE324D4583D5,51.96,1,0.0,51.96,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,161216233231~64B016BA-D9BA-49C7-8726-4BCB6516A11D,FAMILY_DINING,Edmonton,CA
7121730,2024-12-31 17:00:34,49.35,11.59,0.0,49.35,0.00,0.0,241231170034~A9B4401D-26DC-4FC5-A4F9-0261D950DB3B,2024-12-31,59,...,241231165935~27989697-086C-4F7C-8AB2-8C0A8B2CF3A7,49.35,1,0.0,49.35,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA


In [14]:
df_merged['order_take_out_type_label'].unique()
# 'dinein' = 0, 'takeout' = 1, 'delivery' = 2, 'bartab' = 3, 'onlineorder' = 4

array(['dinein', 'takeout', 'delivery', 'bartab', 'onlineorder'],
      dtype=object)

In [15]:
import holidays

def add_columns(df):
    holiday_calendars = {
        'US': holidays.US(),
        'CA': holidays.CA()
    }

    # Ensure bill_paid_at_local is in datetime format
    df['bill_paid_at_local'] = pd.to_datetime(df['bill_paid_at_local'], errors='coerce')

    def is_holiday(row):
        country_code = row['country']
        date = row['bill_paid_at_local']
        
        # Default to 0 if country is missing or not in the dictionary
        if country_code not in holiday_calendars or pd.isna(date):
            return 0

        return 1 if date in holiday_calendars[country_code] else 0
    
    df['holiday'] = df.apply(is_holiday, axis=1)
    df['time'] = df['bill_paid_at_local'].dt.strftime('%H:%M:%S')  # Extract time as string
    df['day_of_week'] = df['bill_paid_at_local'].dt.dayofweek  # Monday=0, Sunday=6
    df['hour_of_day'] = df['bill_paid_at_local'].dt.hour
    df['is_weekend'] = df['bill_paid_at_local'].dt.dayofweek.isin([5, 6]).astype(int)

    df.rename(columns={'payment_count': 'num_people'}, inplace=True)

    # Avoid division errors: Fill NaN values and avoid division by zero
    df['payment_per_person'] = df['payment_amount'] / df['num_people'].replace(0, np.nan)
    df['payment_per_person'] = df['payment_per_person'].fillna(0)

    return df

df = add_columns(df_merged)
df


,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,...,waiter_uuid,concept,city,country,holiday,time,day_of_week,hour_of_day,is_weekend,payment_per_person
0,2024-07-02 20:00:53,102.53,0.00,0.0,95.00,7.53,0.0,240702200053~8792ADCC-545E-4AF3-9836-9C428ED8285C,2024-07-02,5071,...,200421150750~37D0C51E-EC4F-4EA1-B549-D223DA183ABD,BAR,Grand Forks,US,0,20:00:53,1,20,0,124.53
1,2024-07-03 21:43:39,5.38,5.00,0.0,5.00,0.38,0.0,240703214339~B06B2A2F-B1A0-44C0-920A-242F41B58BD2,2024-07-03,6968,...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US,0,21:43:39,2,21,0,5.38
2,2024-07-03 18:23:56,141.90,0.00,0.0,132.00,9.90,0.0,240703182356~D082FE98-841C-4EAB-B15E-BEA41FA1CDF0,2024-07-03,2011,...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US,0,18:23:56,2,18,0,171.90
3,2024-07-03 19:33:21,63.77,0.00,0.0,59.00,4.77,0.0,240703193321~273CBB07-FAB0-49B7-A445-8474F9A4570F,2024-07-03,5480,...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US,0,19:33:21,2,19,0,73.77
4,2024-07-05 21:01:14,210.65,0.00,0.0,195.00,15.65,0.0,240705210114~D423B906-1AEE-4DB8-84BA-234267A14E05,2024-07-05,4773,...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US,0,21:01:14,4,21,0,252.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7121727,2024-12-29 20:32:54,82.42,0.00,0.0,82.42,0.00,0.0,241229203254~464AA06D-F986-4C51-8A53-A7B8362072D6,2024-12-29,4382,...,220831171122~1455DBA8-211C-4281-BF27-8698365CA3EB,FAMILY_DINING,Edmonton,CA,0,20:32:54,6,20,1,82.42
7121728,2024-12-30 20:36:19,87.92,0.00,0.0,87.92,0.00,0.0,241230203619~37348A86-98D1-4113-8EDD-06A3CF61A85B,2024-12-30,5244,...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA,0,20:36:19,0,20,0,87.92
7121729,2024-12-31 14:24:23,51.96,0.00,0.0,51.96,0.00,0.0,241231142423~9D2E9295-9E59-4E76-8242-D230C9FD2494,2024-12-31,963,...,161216233231~64B016BA-D9BA-49C7-8726-4BCB6516A11D,FAMILY_DINING,Edmonton,CA,0,14:24:23,1,14,0,51.96
7121730,2024-12-31 17:00:34,49.35,11.59,0.0,49.35,0.00,0.0,241231170034~A9B4401D-26DC-4FC5-A4F9-0261D950DB3B,2024-12-31,59,...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA,0,17:00:34,1,17,0,49.35


In [16]:
df.columns

Index(['bill_paid_at_local', 'bill_total_billed',
       'bill_total_discount_item_level', 'bill_total_gratuity',
       'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'bill_uuid',
       'business_date', 'order_duration_seconds', 'order_seated_at_local',
       'order_closed_at_local', 'order_take_out_type_label', 'order_uuid',
       'payment_amount', 'num_people', 'payment_total_tip',
       'sales_revenue_with_tax', 'venue_xref_id', 'waiter_uuid', 'concept',
       'city', 'country', 'holiday', 'time', 'day_of_week', 'hour_of_day',
       'is_weekend', 'payment_per_person'],
      dtype='object')

In [17]:
# Ensure datetime format
df['bill_paid_at_local'] = pd.to_datetime(df['bill_paid_at_local'])

# Aggregate to hourly level
df_hourly = df.groupby(['venue_xref_id', df['bill_paid_at_local'].dt.floor('H')]).agg({
    'bill_total_net': 'sum',  # Sum up earnings per hour
    'bill_total_billed': 'sum',  # Total billed amount per hour
    'bill_total_discount_item_level': 'sum',  # Sum of discounts per hour
    'bill_total_gratuity': 'sum',  # Total gratuity per hour
    'bill_total_tax': 'sum',  # Total tax per hour
    'bill_total_voided': 'sum',  # Total voided bills per hour
    'payment_amount': 'sum',  # Total payment amount per hour
    'num_people': 'sum',  # Total number of people per hour
    'payment_total_tip': 'sum',  # Total tip per hour
    'sales_revenue_with_tax': 'sum',  # Total revenue including tax per hour
    'holiday': 'max',  # If at least one row was a holiday, keep it as 1
    'day_of_week': 'first',  # Day of week stays the same per hour
    'hour_of_day': 'first',  # Hour stays the same
    'is_weekend': 'first',  # Weekend status stays the same
    'payment_per_person': 'mean'  # Average payment per person
}).reset_index()

# df_city_mapping = df[['venue_xref_id', 'city']].drop_duplicates() # For weather data
# df_hourly = df_hourly.merge(df_city_mapping, on='venue_xref_id', how='left') # For weather data

df_hourly


/var/folders/f9/nncdwn0j071dw_nr859z0z7h0000gn/T/ipykernel_70251/2567950650.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.groupby(['venue_xref_id', df['bill_paid_at_local'].dt.floor('H')]).agg({


,venue_xref_id,bill_paid_at_local,bill_total_net,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_tax,bill_total_voided,payment_amount,num_people,payment_total_tip,sales_revenue_with_tax,holiday,day_of_week,hour_of_day,is_weekend,payment_per_person
0,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 11:00:00,37.74,42.49,0.0,0.00,4.75,0.0,46.49,3,4.00,42.49,0,1,11,0,15.496667
1,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 12:00:00,162.20,182.82,0.0,0.00,20.62,0.0,191.50,9,8.68,182.82,0,1,12,0,21.277778
2,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 13:00:00,84.96,95.86,0.0,0.00,10.90,2.0,95.86,7,0.00,95.86,0,1,13,0,13.694286
3,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 14:00:00,22.73,25.69,0.0,0.00,2.96,0.0,25.69,2,0.00,25.69,0,1,14,0,12.845000
4,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 15:00:00,46.35,52.38,0.0,0.00,6.03,0.0,52.38,4,0.00,52.38,0,1,15,0,13.095000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683308,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 19:00:00,1229.50,1330.94,26.0,0.00,101.44,0.0,1650.94,5,320.00,1330.94,0,1,19,0,330.188000
683309,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 20:00:00,2828.25,3061.59,46.0,104.35,233.34,204.5,3668.12,19,502.18,3061.59,0,1,20,0,183.406000
683310,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 21:00:00,362.00,391.87,73.0,40.60,29.87,0.0,467.97,5,35.50,391.87,0,1,21,0,93.594000
683311,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 22:00:00,606.00,656.00,29.0,0.00,50.00,0.0,726.00,3,70.00,656.00,0,1,22,0,181.500000


In [18]:
# Extract test df_hourly and train df
test_venues = df_hourly['venue_xref_id'].drop_duplicates().head(10).values
df_test = df_hourly[df_hourly['venue_xref_id'].isin(test_venues)].copy()
df_train = df_hourly[~df_hourly['venue_xref_id'].isin(test_venues)].copy()

df_test.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

print(f"Original DataFrame size: {df_hourly.shape}")
print(f"Training DataFrame size: {df_train.shape}")
print(f"Test DataFrame size: {df_test.shape}")


Original DataFrame size: (683313, 17)
Training DataFrame size: (668908, 17)
Test DataFrame size: (14405, 17)


# ML Prediction Model

In [19]:
df_train

,venue_xref_id,bill_paid_at_local,bill_total_net,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_tax,bill_total_voided,payment_amount,num_people,payment_total_tip,sales_revenue_with_tax,holiday,day_of_week,hour_of_day,is_weekend,payment_per_person
0,0879d0bfe26c46e940b6e90d2d0bdd1a46f9090b5b6b5c...,2024-07-01 13:00:00,346.00,363.30,0.0,0.00,17.30,0.0,428.69,1,65.39,363.30,1,0,13,0,428.690000
1,0879d0bfe26c46e940b6e90d2d0bdd1a46f9090b5b6b5c...,2024-07-01 14:00:00,90.00,94.50,0.0,0.00,4.50,0.0,115.29,1,20.79,94.50,1,0,14,0,115.290000
2,0879d0bfe26c46e940b6e90d2d0bdd1a46f9090b5b6b5c...,2024-07-01 15:00:00,201.00,220.45,0.0,0.00,19.45,0.0,261.24,2,40.79,220.45,1,0,15,0,130.620000
3,0879d0bfe26c46e940b6e90d2d0bdd1a46f9090b5b6b5c...,2024-07-01 16:00:00,356.00,380.80,13.0,0.00,24.80,0.0,453.01,6,72.21,380.80,1,0,16,0,77.507000
4,0879d0bfe26c46e940b6e90d2d0bdd1a46f9090b5b6b5c...,2024-07-01 17:00:00,508.00,560.90,25.0,59.00,52.90,0.0,661.39,6,41.49,560.90,1,0,17,0,94.484286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668903,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 19:00:00,1229.50,1330.94,26.0,0.00,101.44,0.0,1650.94,5,320.00,1330.94,0,1,19,0,330.188000
668904,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 20:00:00,2828.25,3061.59,46.0,104.35,233.34,204.5,3668.12,19,502.18,3061.59,0,1,20,0,183.406000
668905,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 21:00:00,362.00,391.87,73.0,40.60,29.87,0.0,467.97,5,35.50,391.87,0,1,21,0,93.594000
668906,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 22:00:00,606.00,656.00,29.0,0.00,50.00,0.0,726.00,3,70.00,656.00,0,1,22,0,181.500000


## Earning Prediction Model

In [28]:
# Convert datetime
df_train['bill_paid_at_local'] = pd.to_datetime(df_train['bill_paid_at_local'])
df_train = df_train.sort_values('bill_paid_at_local')

# Define features & target
features = [
    'bill_total_net', 'bill_total_billed', 'bill_total_discount_item_level',
    'bill_total_gratuity', 'bill_total_tax', 'bill_total_voided',
    'payment_amount', 'num_people', 'payment_total_tip', 'sales_revenue_with_tax',
    'holiday', 'day_of_week', 'hour_of_day', 
    'is_weekend', 'payment_per_person'
]
target = 'bill_total_net'  # ✅ Predicting **hourly earnings**, NOT weekly sum!

# Split Train & Test
latest_date = df_train['bill_paid_at_local'].max()
cutoff_date = latest_date - timedelta(days=7)

train_data = df_train[df_train['bill_paid_at_local'] < cutoff_date]
test_data = df_train[df_train['bill_paid_at_local'] >= cutoff_date]

X_train, y_train = train_data[features], train_data[target]
X_test, y_test = test_data[features], test_data[target]

# ✅ Scale Features (Same as Before)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Train New `ep_model`
from xgboost import XGBRegressor

ep_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

ep_model.fit(X_train, y_train)

# ✅ Save Model & Scaler
with open("ep_model0.pkl", "wb") as f:
    pickle.dump(ep_model, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)



In [31]:
# Train XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, early_stopping_rounds=20,
    tree_method='hist', random_state=42
)

print("Training XGBoost model on venue sequences...")
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=50)
print("✅ Training complete.")

# Save the trained model
with open("ep_model0.pkl", "wb") as f:
    pickle.dump(xgb_model, f)
print("✅ Model saved.")

Training XGBoost model on venue sequences...
[0]	validation_0-rmse:546.98950
[50]	validation_0-rmse:160.58565
[100]	validation_0-rmse:141.34915
[150]	validation_0-rmse:129.58759
[194]	validation_0-rmse:125.89781
✅ Training complete.
✅ Model saved.


In [32]:
# Evaluate Model Performance
y_pred = xgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred) 
r2 = r2_score(y_test, y_pred)

df_results = pd.DataFrame([{"MAE": mae, "RMSE": rmse, "R² Score": r2}])
print(df_results)

print("✅ Model trained & evaluated. Ready to predict future earnings.")

        MAE        RMSE  R² Score
0  7.341161  125.428866  0.951686
✅ Model trained & evaluated. Ready to predict future earnings.


## Potential Earning Prediction Model

In [23]:
df_hourly['potential_earnings'] = df_hourly.groupby(
    ['venue_xref_id', 'day_of_week', 'hour_of_day']
)['bill_total_net'].transform(lambda x: x.quantile(0.90))

df_hourly['potential_vs_actual'] = df_hourly['potential_earnings'] - df_hourly['payment_amount']

df_hourly

,venue_xref_id,bill_paid_at_local,bill_total_net,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_tax,bill_total_voided,payment_amount,num_people,payment_total_tip,sales_revenue_with_tax,holiday,day_of_week,hour_of_day,is_weekend,payment_per_person,potential_earnings,potential_vs_actual
0,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 11:00:00,37.74,42.49,0.0,0.00,4.75,0.0,46.49,3,4.00,42.49,0,1,11,0,15.496667,222.115,175.625
1,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 12:00:00,162.20,182.82,0.0,0.00,20.62,0.0,191.50,9,8.68,182.82,0,1,12,0,21.277778,220.795,29.295
2,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 13:00:00,84.96,95.86,0.0,0.00,10.90,2.0,95.86,7,0.00,95.86,0,1,13,0,13.694286,133.150,37.290
3,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 14:00:00,22.73,25.69,0.0,0.00,2.96,0.0,25.69,2,0.00,25.69,0,1,14,0,12.845000,114.786,89.096
4,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,2024-07-02 15:00:00,46.35,52.38,0.0,0.00,6.03,0.0,52.38,4,0.00,52.38,0,1,15,0,13.095000,100.920,48.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683308,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 19:00:00,1229.50,1330.94,26.0,0.00,101.44,0.0,1650.94,5,320.00,1330.94,0,1,19,0,330.188000,1285.650,-365.290
683309,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 20:00:00,2828.25,3061.59,46.0,104.35,233.34,204.5,3668.12,19,502.18,3061.59,0,1,20,0,183.406000,1791.800,-1876.320
683310,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 21:00:00,362.00,391.87,73.0,40.60,29.87,0.0,467.97,5,35.50,391.87,0,1,21,0,93.594000,1063.500,595.530
683311,ffe3cf683c3b3d6f7c1ae13deb93a0bdb9f87ffb3b658b...,2024-12-31 22:00:00,606.00,656.00,29.0,0.00,50.00,0.0,726.00,3,70.00,656.00,0,1,22,0,181.500000,412.000,-314.000


In [24]:
features = [
    'bill_total_net', 'bill_total_billed', 'bill_total_discount_item_level',
    'bill_total_gratuity', 'bill_total_tax', 'bill_total_voided',
    'payment_amount', 'num_people', 'payment_total_tip', 'sales_revenue_with_tax',
    'holiday', 'day_of_week', 'hour_of_day', 
    'is_weekend', 'payment_per_person'
]

target = 'potential_earnings'

# Train-Test Split (Train on past data, Test on recent data)
df_hourly['bill_paid_at_local'] = pd.to_datetime(df_hourly['bill_paid_at_local'])
df_hourly = df_hourly.sort_values('bill_paid_at_local')  # Sort by time

train_data = df_hourly[df_hourly['bill_paid_at_local'] < df_hourly['bill_paid_at_local'].max() - pd.Timedelta(days=7)]
test_data = df_hourly[df_hourly['bill_paid_at_local'] >= df_hourly['bill_paid_at_local'].max() - pd.Timedelta(days=7)]

X_train, y_train = train_data[features], train_data[target]
X_test, y_test = test_data[features], test_data[target]

# Assign Weights Based on Recency
def assign_weights(date, max_date):
    days_old = (max_date - date).days
    if days_old <= 30:
        return 1.5  # Recent data gets higher weight
    elif days_old <= 60:
        return 1.2  # Medium-weight
    else:
        return 1.0  # Older data gets lower weight

train_data['weights'] = train_data['bill_paid_at_local'].apply(lambda x: assign_weights(x, train_data['bill_paid_at_local'].max()))


/var/folders/f9/nncdwn0j071dw_nr859z0z7h0000gn/T/ipykernel_70251/436727352.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['weights'] = train_data['bill_paid_at_local'].apply(lambda x: assign_weights(x, train_data['bill_paid_at_local'].max()))


In [25]:
# Train XGBoost Model (With Weights)
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Regression task
    n_estimators=200,              # Number of trees
    learning_rate=0.05,            # Step size shrinkage
    max_depth=6,                   # Maximum depth of trees
    subsample=0.8,                 # Randomly sample training data
    colsample_bytree=0.8,          # Randomly sample features
    random_state=42
)

xgb_model.fit(X_train, y_train, sample_weight=train_data['weights'])

with open("pp_model1.pkl", "wb") as f:
    pickle.dump(xgb_model, f)
print("✅ Model saved as 'pp_model1.pkl'.")

✅ Model saved as 'pp_model1.pkl'.


In [26]:
# Make Predictions
with open("pp_model.pkl", "rb") as f:
    xgb_model = pickle.load(f)
print("✅ Model loaded successfully.")
y_pred = xgb_model.predict(X_test)

# Evaluate Model Performance
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(root_mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

✅ Model loaded successfully.


ValueError: feature_names mismatch: ['bill_total_net', 'bill_total_billed', 'bill_total_discount_item_level', 'bill_total_gratuity', 'bill_total_tax', 'bill_total_voided', 'payment_amount', 'num_people', 'payment_total_tip', 'sales_revenue_with_tax', 'holiday', 'day_of_week', 'hour_of_day', 'is_weekend', 'payment_per_person', 'max_temperature', 'min_temperature', 'avg_temperature', 'precipitation_sum', 'avg_humidity'] ['bill_total_net', 'bill_total_billed', 'bill_total_discount_item_level', 'bill_total_gratuity', 'bill_total_tax', 'bill_total_voided', 'payment_amount', 'num_people', 'payment_total_tip', 'sales_revenue_with_tax', 'holiday', 'day_of_week', 'hour_of_day', 'is_weekend', 'payment_per_person']
expected avg_humidity, min_temperature, max_temperature, avg_temperature, precipitation_sum in input data

# Test Data Extraction

In [ ]:
# test_df = df[df['venue_xref_id'] == 'ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9d89b7513f81fc00bed']
# test_df.to_csv("test_venue.csv", index=False)